# **DAILY MACRO RELATIVE VALUE TRADE FRAMEWORK**
### Author:  Deborah Akintoye
#### **Purpose:** Convert one macro headline into a backtestable relative-value trade with clear horizon, factor sensitivities, execution logic, and weekly performance tracking.

---


## 1. Trade Metadata & Idea Structuring

**Date:    19-Jan-2026**

**News Source / Catalyst:** EU readies €93bn tariffs in retaliation for Trump’s Greenland threat

**Markets (s) Impacted:** FX (G10/EM), FI (Euro-Periphery vs. EM spreads)

**Trade Taxonomy:** Macro RV Baskets / Policy Divergence

**Trade Idea (1 sentence):**
> Long a basket of INR, IDR, and PHP (Resilient Asian Carry) against a Short basket of NOK, SEK, and GBP (Greenland/Aric Tariff Targets). If tariffs escalate → NOK/SEK/GBP weaken relative to INR/IDR/PHP via oil beta, current account pressures, and risk premium.

**Client/Desk Pitch (30 seconds):**
> We are shorting the "Western Alliance Fissure." While Trump targets the UK and Scandis with 10% tariffs over the Greenland dispute, and the EU prepares €93bn in retaliation, the structural growth stories in South/Southeast Asia remain decoupled from Arctic geopolitics. This trade captures the "Geopolitical Risk Premium" shift from the North Atlantic to a stable, carry-rich Asian basket.

**Why Now:**

The "Davos Countdown." With the Feb 1 tariff deadline looming, we expect a volatility "squeeze" in Scandi and UK markets as negotiators head to Switzerland.

---

## 2. Macro Hypothesis & Instruments

### **Macro Hypothesis:**

The "Greenland Discount" will metastasize across the currencies of countries explicitly named in Trump's tariff threat (UK, Norway, Sweden). Conversely, India, Indonesia, and the Philippines are net energy importers that benefit from trade-war-induced oil price suppression and have high real rates that provide a "carry buffer."

**Long Leg (Tickers + Rationale):**
- INR (USDINR=X), IDR (USDIDR=X), PHP (USDPHP=X); Low correlation to Arctic security; benefit from structural capital inflows; high domestic demand insulation

**Short Leg (Tickers + Rationale):**
- NOK (USDNOK=X), SEK (USDSEK=X), GBP (GBPUSD=X); Direct tariff targets; NOK/SEK liquidity risk; GBP vulnerability to trade negotiations

**Weights:** Beta-Neutral / Vol-Scaled

**Allocation**: We apply a $3:1$ ratio on notional. Because EM FX volatility is artificially suppressed by central bank intervention (RBI, BI), we need higher notional in the Long leg to match the high-beta daily swings of the Scandis.

---

## 3. Factor Sensitivity & Horizon Alignment

| Factor                | Strength (S/M/W) | Notes |
|----------------------|------------------|-------|
| USD / FX             |        M          |  The basket is USD-neutral; we are trading the Cross-Rate proxy     |
| Oil Beta             |        H          |   Long leg = Short Oil; Short leg = Long Oil (specifically NOK)    |
| Rates Duration       |        M          |   Positive carry (EM yields > Scandi/UK yields)    |
| Growth vs Value      |        M          |   Favors domestic service-led growth (EM) over export-led (Scandi)    |
| Risk-On/Off          |        M          |   Defensive in a localsed "Atlantic Trade War" scenario    |
| China/EM Beta        |        S          |   Long leg captures the "Neutral EM" safety trade    |
| Volatility Regime    |        S          |  Betting on a vol-spike in Northern Europe vs. stability in Asia     |

**Catalyst Type:** Policy / Regime Shift

**Realisation Window:** 14 - 21 days (leading to 01-Feb Implementation)

**Holding Period Selected:**  Tactical (Davos meeting duration)

**Alignment Score:** Good

---


## 4. Risk Controls & Stop Logic

**Position Sizing:** 5.0% of portfolio notional (Aggregate basket)

**Stop-Loss:** -2.8% on the basket spread

**Take-Profit (optional):**  7.5% (Triggered if EU formally activates the "Anti-Coercion Instrument")

**Instrument Contamination:**   We avoid EUR and CHF in the short leg because they act as safe-havens during European panics, which would hedge our short unfairly. We use USD as the denominator for all to maintain a "Synthetic Cross" structure.

**Failure Mode (tick):**
- [ ] Time
- [ ] Price
- [X] Information

---


# **PYTHON IMPLEMENTATION**

In [27]:
# --- Python Imports ---

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: f'{x:.4f}')


In [28]:
# --- Parameters ---

long_tickers  = ["INR=X", "IDR=X", "PHP=X"]   # EM Carry / Asia
short_tickers = ["NOK=X", "SEK=X", "GBP=X"]   # Scandi + UK (Tariff Targets)


start_date = "2020-01-01"                         # Backtest window
holding_days = 14                               # days
stop_loss = -0.028                                 # -2% stop
take_profit = 0.075                                # optional e.g. +0.03 for +3%


In [29]:
# --- Data Download ---

tickers = long_tickers + short_tickers
data = yf.download(tickers, start=start_date)["Close"].dropna().ffill()


log_rets = np.log(data / data.shift(1)).dropna()

# --- Compute Volatility (20D realised) ---

vols = log_rets.rolling(20).std().mean()  # mean realized vol per ticker
inv_vols = 1 / vols

# --- Normalised Weights ---

weights_long  = inv_vols[long_tickers]  / inv_vols[long_tickers].sum()
weights_short = inv_vols[short_tickers] / inv_vols[short_tickers].sum()

print("Long Basket Weights:")
print(weights_long.round(3))
print("\nShort Basket Weights:")
print(weights_short.round(3))

long_basket_rets  = (log_rets[long_tickers].mul(weights_long)).sum(axis=1)
short_basket_rets = (log_rets[short_tickers].mul(weights_short)).sum(axis=1)

strategy_rets = long_basket_rets - short_basket_rets



/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_9717/2359773586.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date)["Close"].dropna().ffill()
[*********************100%***********************]  6 of 6 completed

Long Basket Weights:
Ticker
INR=X   0.4730
IDR=X   0.2240
PHP=X   0.3030
dtype: float64

Short Basket Weights:
Ticker
NOK=X   0.2300
SEK=X   0.3370
GBP=X   0.4330
dtype: float64



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:395: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


In [30]:
# --- Backtest Simulation ---

trade_pnl = []
trade_dates = []

dates = strategy_rets.index
i = 0
n = len(strategy_rets)

while i < n - holding_days:
    entry_date = dates[i]
    pnl = 0.0
    stopped = False
    
    for j in range(holding_days):
        daily_ret = strategy_rets.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        # Stop Loss
        if pnl <= stop_loss:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
        
        # Take Profit
        if take_profit is not None and pnl >= take_profit:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
    
    if not stopped:
        trade_pnl.append(pnl)
        trade_dates.append(entry_date)
    
    i += holding_days

# --- Results ---
trade_results = pd.DataFrame({"Trade Date": trade_dates, "PnL": trade_pnl})

summary = pd.Series({
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Best Trade": trade_results["PnL"].max(),
    "Worst Trade": trade_results["PnL"].min(),
    "Stop-Out Frequency": (trade_results["PnL"] <= stop_loss).mean()
})

print("\n--- Strategy Summary ---")
print(summary.round(4))


--- Strategy Summary ---
Total Trades         112.0000
Win Rate               0.4643
Average PnL            0.0019
Best Trade             0.0869
Worst Trade           -0.0369
Stop-Out Frequency     0.0804
dtype: float64


## Interpretation & Trader Notes

**Absolute Metrics:**
- Total Trades: 112.0000
- Win Rate: 0.4643
- Avg PnL: 0.0019
- Best: 0.0869
- Worst: -0.0369
- Stop-Outs: 0.0804

**What went right:**
- Low stop-out rate (good)
- Positive expectancy (good)
- Not skewed by big outliers (healthy)
- Moderately noisy win-rate (normal for FX RV)

**What went wrong:**
- Add payoff mapping sentence
- Tighten catalyst → horizon → exit framing

**Factor that actually drove results:**

| Factor    |	Role    |
------------|-------------|
| USD FX    |	secondary (basket USD-neutral)  |
| Oil beta  |	primary driver via NOK  |
| Rates carry   |	supports INR/IDR/PHP    |
| Risk-on/off   |	moderate regime passthrough |



**Would I trade this live?** 
> Yes, as a small tactical expression. I like the theme, but I’d size small and maybe overlay long CL puts or short EURNOK to isolate oil more cleanly.


---

# **BLOOMBERG-STYLE TRADE BLOTTER**

In [31]:
import uuid
from datetime import datetime

def generate_trade_id(prefix="RV"):
    return f"{prefix}-{uuid.uuid4().hex[:8].upper()}"


In [32]:
# --- Computing Synthetic Basket Price Series ---

basket_long = (data[long_tickers] * weights_long).sum(axis=1)
basket_short = (data[short_tickers] * weights_short).sum(axis=1)

basket_spread = basket_long - basket_short


In [33]:
# --- Backtesting Loop Modified for Blotter ---

blotter_rows = []

i = 0
n = len(strategy_returns)
dates = strategy_returns.index

while i < n - holding_period:
    entry_date = dates[i]
    exit_date = dates[min(i + holding_period - 1, n - 1)]
    
    entry_price = basket_spread.loc[entry_date]
    
    pnl = 0.0
    stopped = False
    trade_status = "CLOSED"
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            exit_date = dates[i + j]
            trade_status = "STOPPED"
            stopped = True
            break
    
    if not stopped and take_profit is not None and pnl >= take_profit:
        exit_date = dates[i + j]
        trade_status = "TAKE_PROFIT"
    
    exit_price = basket_spread.loc[exit_date]
    holding_days = (exit_date - entry_date).days
    
    row = {
        "TradeID": generate_trade_id(prefix="RV"),
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Strategy": "Macro RV Spread",
        "Taxonomy": trade_taxonomy if 'trade_taxonomy' in globals() else "RV Macro Spread",
        "LongLeg": ", ".join(long_tickers),
        "ShortLeg": ", ".join(short_tickers),
        "EntryDate": entry_date.strftime("%Y-%m-%d"),
        "ExitDate": exit_date.strftime("%Y-%m-%d"),
        "EntryPrice": entry_price,
        "ExitPrice": exit_price,
        "GrossPnL(%)": pnl * 100,
        "StopLoss(%)": stop_loss * 100,
        "TakeProfit(%)": take_profit * 100 if take_profit else None,
        "Status": trade_status,
        "HoldingPeriod": holding_days,
        "Notes": "",  # Optional notes field
    }
    blotter_rows.append(row)
    
    i += holding_period

blotter = pd.DataFrame(blotter_rows)
blotter.tail()


,TradeID,Timestamp,Strategy,Taxonomy,LongLeg,ShortLeg,EntryDate,ExitDate,EntryPrice,ExitPrice,GrossPnL(%),StopLoss(%),TakeProfit(%),Status,HoldingPeriod,Notes
107,RV-9B7762F0,2026-01-19 07:01:37,Macro RV Spread,RV Macro Spread,"INR=X, IDR=X, PHP=X","NOK=X, SEK=X, GBP=X",2025-10-03,2025-10-22,3775.2806,3774.3800,1.2988,-2.8000,7.5000,CLOSED,19,
108,RV-DF6BFE05,2026-01-19 07:01:37,Macro RV Spread,RV Macro Spread,"INR=X, IDR=X, PHP=X","NOK=X, SEK=X, GBP=X",2025-10-23,2025-11-11,3773.8189,3795.3023,0.1929,-2.8000,7.5000,CLOSED,19,
109,RV-A10628C3,2026-01-19 07:01:37,Macro RV Spread,RV Macro Spread,"INR=X, IDR=X, PHP=X","NOK=X, SEK=X, GBP=X",2025-11-12,2025-12-01,3796.3331,3783.1873,-0.3492,-2.8000,7.5000,CLOSED,19,
110,RV-CBF154D1,2026-01-19 07:01:37,Macro RV Spread,RV Macro Spread,"INR=X, IDR=X, PHP=X","NOK=X, SEK=X, GBP=X",2025-12-02,2025-12-19,3777.4685,3794.9470,-1.3330,-2.8000,7.5000,CLOSED,17,
111,RV-F5EC864F,2026-01-19 07:01:37,Macro RV Spread,RV Macro Spread,"INR=X, IDR=X, PHP=X","NOK=X, SEK=X, GBP=X",2025-12-22,2026-01-12,3793.1047,3826.8708,-1.0906,-2.8000,7.5000,CLOSED,21,


In [34]:
blotter.to_csv("trade_blotter.csv", index=False)